# Predictive Traffic Accident Model

## Analysis of 2021 NHTSA FARS Data for Fatality Prediction and Pattern Recognition

## Project Overview

**Objective**: This project analyzes traffic accident data from 2021 to predict fatalities and identify patterns in accident occurrences using machine learning techniques.

**Goals**:
- Predict the number of fatalities in traffic accidents using supervised learning
- Identify clusters and patterns in accident data using unsupervised learning  
- Extract actionable insights about contributing factors to traffic accidents

### Dataset Description

The dataset consists of comprehensive traffic accident records from the **Fatality Analysis Reporting System (FARS)** for 2021, maintained by the National Highway Traffic Safety Administration (NHTSA). 

**Data Sources**:
- **ACC_AUX.CSV**: Accident-level data including location, time, weather, and crash characteristics
- **VEH_AUX.CSV**: Vehicle-level data including vehicle type, model year, and driver information
- **PER_AUX.CSV**: Person-level data including age, injury severity, and demographic information

The merged dataset contains detailed information about each accident including:
- Case numbers and geographical information (state, county)
- Temporal factors (time of day, day of week)
- Vehicle characteristics (type, model year)
- Person demographics and injury details
- Environmental conditions and contributing factors

## 1. Environment Setup and Dependencies

### Python Libraries Used

**Data Manipulation & Analysis**:
- **Pandas**: Primary tool for data manipulation, merging datasets, and exploratory data analysis
- **NumPy**: Numerical computations and array operations

**Machine Learning**:
- **Scikit-learn**: Comprehensive machine learning library providing:
  - Linear Regression for supervised learning (fatality prediction)
  - K-Means clustering for unsupervised pattern discovery
  - Cross-validation tools for model evaluation
  - Preprocessing utilities (StandardScaler, OneHotEncoder)

**Data Visualization**:
- **Matplotlib**: Core plotting library for creating static visualizations
- **Seaborn**: Statistical visualization library built on matplotlib for enhanced plotting
- **Plotly**: Interactive visualization library for dynamic plots and dashboards

**Statistical Analysis**:
- **SciPy**: Statistical functions and hypothesis testing tools

### Methodology Overview

**Data Integration Approach**: 
- Merge three separate FARS datasets using case numbers as primary keys
- Handle missing values and data quality issues systematically
- Create meaningful categorical mappings for encoded variables

**Feature Engineering Strategy**:
- Transform categorical codes into interpretable labels
- Create combined features to capture complex relationships
- Standardize numerical features for machine learning algorithms

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Plotly Packages (optional - comment out if not available)
try:
    from plotly import tools
    import plotly.figure_factory as ff
    import plotly.graph_objs as go
    from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
    init_notebook_mode(connected=True)
    PLOTLY_AVAILABLE = True
except ImportError:
    print("Plotly not available - using matplotlib/seaborn for all visualizations")
    PLOTLY_AVAILABLE = False

# Statistical Libraries
from scipy.stats import norm
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D

# Folium for mapping (optional - comment out if not available)
try:
    import folium
    from folium.plugins import HeatMap
    FOLIUM_AVAILABLE = True
except ImportError:
    print("Folium not available - skipping map visualizations")
    FOLIUM_AVAILABLE = False

%matplotlib inline

## 2. Data Loading and Initial Exploration

This section loads the three FARS datasets and performs initial data quality assessment:

- **Data Quality Check**: Identify missing values and data inconsistencies
- **Data Integration**: Merge datasets using case numbers as primary keys
- **Initial Statistics**: Basic descriptive statistics for understanding data distribution

In [2]:
# reading the files 

df_veh_aux = pd.read_csv('../datasets/VEH_AUX.CSV')
df_veh_aux.head()

In [3]:
# check if there are missing values 

missing_values = df_veh_aux.isnull().sum()
print(missing_values)

In [4]:
# drop missing value columns 
df_veh_aux = df_veh_aux.drop(columns=['A_IMP2'])
df_veh_aux.head()

In [5]:
df_veh_aux.describe()

In [6]:
# reading the file and print the head 

df_per_aux = pd.read_csv('../datasets/PER_AUX.CSV')
df_per_aux.head()

In [ ]:
# Check for missing values in vehicle dataset
missing_values = df_veh_aux.isnull().sum()
print("Missing values in Vehicle dataset:")
print(missing_values)

In [8]:
df_per_aux.describe()

In [9]:
df_acc_aux = pd.read_csv('../datasets/ACC_AUX.CSV')
df_acc_aux.head()

In [ ]:
# Check for missing values in accident dataset
missing_values = df_acc_aux.isnull().sum()
print("Missing values in Accident dataset:")
print(missing_values)

In [11]:
df_acc_aux.describe()

In [ ]:
# Verify case number consistency across datasets
print("Case number samples from each dataset:")
print("Accident dataset ST_CASE:", df_acc_aux['ST_CASE'].head())
print("Person dataset ST_CASE:", df_per_aux['ST_CASE'].head())
print("Vehicle dataset ST_CASE:", df_veh_aux['ST_CASE'].head())

In [ ]:
# Data Integration: Merge the three datasets
print("Merging datasets...")

# Merge vehicle and person data first
merged_veh_per = pd.merge(df_veh_aux, df_per_aux, on='ST_CASE', how='inner')

# Then merge with accident data
merged_all = pd.merge(df_acc_aux, merged_veh_per, on='ST_CASE', how='inner')

# Check for duplicate rows and columns
duplicate_rows = merged_all.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)

duplicate_columns = merged_all.columns.duplicated().sum()
print("Number of duplicate columns:", duplicate_columns)

# Remove duplicates if present
if duplicate_rows > 0:
    merged_all = merged_all.drop_duplicates()
    print("Duplicate rows removed.")

# Remove duplicate columns
merged_all = merged_all.loc[:,~merged_all.columns.duplicated()]

print(f"Final merged dataset shape: {merged_all.shape}")
print("\nFirst 5 rows of merged dataset:")
print(merged_all.head())

In [16]:
# Compare YEAR_x and 'YEAR_y' to check if they are the same
same_year = (merged_all['YEAR_x'] == merged_all['YEAR_y']).all()
print(f"YEAR_x and YEAR_y are same {same_year}")

# Compare VEH_NO_x and y  
same_veh_no = (merged_all['VEH_NO_x'] == merged_all['VEH_NO_y']).all()
print(f"VEH_NO_x and VEH_NO_y are the same: {same_veh_no}")

# Drop duplicate columns 
if same_year:
    merged_all.drop(columns=['YEAR_y'], inplace=True)
if same_veh_no:
    merged_all.drop(columns=['VEH_NO_y'], inplace=True)

# rename columns 
if not same_year or not same_veh_no:
    merged_all.rename(columns={'YEAR_x': 'Year_Accident', 'YEAR_y': 'Year_Vehicle',
                               'VEH_NO_x': 'Vehicle_No_Accident', 'VEH_NO_y': 'Vehicle_No_Person'}, inplace=True)

# Check the updated dataframe
print(merged_all.head())

In [17]:
merged_all.head()
merged_all.info()

In [18]:
merged_all.describe()

In [19]:
merged_all.head()

## 3. Exploratory Data Analysis and Visualization

This section explores key patterns in the traffic accident data through statistical analysis and visualization.

### Key Insights from Exploratory Analysis

**Temporal Patterns**: 
- **Weekday vs. Weekend Distribution**: Analysis reveals distinct patterns in accident frequency between weekdays and weekends, potentially related to commuter traffic patterns and recreational driving behaviors.

**Demographic Analysis**:
- **Age Group Involvement**: The distribution of crashes across different age groups provides insights into driving experience and risk-taking behaviors. Younger age groups (16-24 years) often show higher involvement rates, suggesting factors like inexperience, risk-taking behavior, or distracted driving.

**Vehicle and Environmental Factors**:
- **Time of Day Impact**: Accident patterns vary significantly by time of day, with implications for targeted safety interventions and traffic management.
- **Vehicle Type Correlation**: Different vehicle types show varying involvement rates in fatal accidents, informing vehicle safety feature development.

In [20]:

# check the describe for these columns 
print(merged_all[['A_TOD', 'A_DOW', 'A_BODY']].describe())

# Mapping of A_TOD to 1, 2 and 3 
time_of_day_mapping = {
    1: 'Daytime',
    2: 'Nighttime',
    3: 'Unknown'
}

# create a new Time_of_Day column and mapping to it 
merged_all['Time_of_Day'] = merged_all['A_TOD'].map(time_of_day_mapping)

# Create a count plot for the distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_all, x='Time_of_Day', palette="viridis")
plt.title('Distribution of Accidents by Time of Day (A_TOD)')
plt.xlabel('Time of Day')
plt.ylabel('Number of Accidents')
plt.show()


#Day of week 
# Mapping of A_DOW
day_of_week_mapping = {
    1: 'Weekday',
    2: 'Weekend',
    3: 'Unknown'
}

# to create a new Day_of_Week column and mapping to it 
merged_all['Day_of_Week'] = merged_all['A_DOW'].map(day_of_week_mapping)

# Create plot for distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_all, x='Day_of_Week', palette="viridis")
plt.title('Distribution of Accidents by Day of Week (A_DOW)')
plt.xlabel('Day of Week')
plt.ylabel('Number of Accidents')
plt.show()











# age group columns and their labels
age_group_columns = {
    'A_D15_19': '15-19 yrs',
    'A_D16_19': '16-19 yrs',
    'A_D15_20': '15-20 yrs',
    'A_D16_20': '16-20 yrs',
    'A_D16_24': '16-24 yrs',
    'A_D21_24': '21-24 yrs',
    'A_D65PLS': '65+ yrs'
}

# Create a DataFrame 
age_group_data = pd.DataFrame()

for col, label in age_group_columns.items():
    temp_df = merged_all[merged_all[col] == 1][col].value_counts().rename_axis('Involvement').reset_index(name='Count')
    temp_df['Age Group'] = label
    age_group_data = pd.concat([age_group_data, temp_df], ignore_index=True)

# a combined chart
plt.figure(figsize=(15, 8))
sns.barplot(data=age_group_data, x='Age Group', y='Count', hue='Involvement', palette="viridis")
plt.title('Distribution of Crashes Involving Different Age Groups')
plt.xlabel('Age Group')
plt.ylabel('Number of Crashes')
plt.show()



# Mapping of A_BODY 
vehicle_body_mapping = {
    1: 'Passenger Car',
    2: 'Light Truck - Pickup',
    3: 'Light Truck - Utility',
    4: 'Light Truck - Van',
    5: 'Light Truck - Other',
    6: 'Large Truck',
    7: 'Motorcycle',
    8: 'Bus',
    9: 'Other'
}

# create a new Vehicle_Type column
merged_all['Vehicle_Type'] = merged_all['A_BODY'].map(vehicle_body_mapping)

# Create a count plot for vehicle type distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=merged_all, x='Vehicle_Type', palette="viridis")
plt.title('Distribution of Vehicle Types in Accidents')
plt.xlabel('Vehicle Type')
plt.ylabel('Number of Accidents')
plt.xticks(rotation=45)
plt.show()


In [21]:


# new features 

print(merged_all[['FATALS', 'A_CRAINJ', 'A_POSBAC', 'A_DIST']].describe())





# Set the style of the plots
sns.set_style("whitegrid")

# Create a histogram for FATALS column
plt.figure(figsize=(10, 6))
# The maximum number of fatalities 
max_fatalities = merged_all['FATALS'].max()
sns.histplot(data=merged_all, x='FATALS', bins=range(1, max_fatalities + 1), kde=False)
plt.title('Histogram of Number of Fatalities (FATALS)')
plt.xlabel('Number of Fatalities')
plt.ylabel('Frequency')
plt.xlim(1, max_fatalities)  # Limiting x-axis to the max number of fatalities
plt.xticks(range(1, max_fatalities + 1))  # Set x-axis ticks for each number of fatalities
plt.show()


# Mapping of A_POSBAC 
bac_test_result_mapping = {
    1: 'Positive BAC',
    2: 'Zero BAC',
    3: 'Unknown BAC'
}

# create a new BAC_Test_Result column
merged_all['BAC_Test_Result'] = merged_all['A_POSBAC'].map(bac_test_result_mapping)

# Create a count plot for distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_all, x='BAC_Test_Result', palette="viridis")
plt.title('Distribution of Accidents by BAC Test Results (A_POSBAC)')
plt.xlabel('BAC Test Result')
plt.ylabel('Number of Accidents')
plt.show()



# Mapping of A_DIST codes
distraction_involvement_mapping = {
    1: 'Distracted Driver Involved',
    2: 'Other Crash'
}

# to create a new Distracted_Driving column
merged_all['Distracted_Driving'] = merged_all['A_DIST'].map(distraction_involvement_mapping)

# Create a count plot for distracted driving distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_all, x='Distracted_Driving', palette="viridis")
plt.title('Distribution of Accidents Involving Distracted Driving (A_DIST)')
plt.xlabel('Distracted Driving Involvement')
plt.ylabel('Number of Accidents')
plt.show()

## 4. Feature Engineering and Data Preprocessing

This section creates meaningful features from the categorical accident data to improve model performance.

### 4.1 Data Preprocessing Steps

**Data Integration Strategy**:
- Merged three FARS datasets (ACC_AUX, VEH_AUX, PER_AUX) using case numbers
- Handled missing values by removing columns with significant missing data (>30%)
- Ensured data consistency across merged datasets

### 4.2 Feature Engineering Approach

**New Feature Creation**:
1. **Combined Risk Factors**: Created composite features combining alcohol involvement and distracted driving
2. **Categorical Mappings**: Transformed coded variables into interpretable categories
3. **Temporal Features**: Enhanced time-based variables for better pattern recognition

**Rationale for Feature Engineering**:
- **Alcohol and Distraction Combination**: Research shows these factors often interact to increase accident severity
- **Age Group Analysis**: Different age demographics show distinct risk patterns requiring separate analysis
- **Categorical Transformation**: Converting numeric codes to meaningful labels improves model interpretability

**Data Transformation**:
- **Categorical to Numeric**: Used one-hot encoding for categorical variables in machine learning models
- **Standardization**: Applied to numerical features for clustering analysis to ensure equal weighting

In [22]:
# python code goes here, you can add more code cells below this one if needed
# to better structure your JN
# include any code you use to do data preparation and feature engineering so the process
# can be replicated


# Creating a new combined feature for alcohol and distraction involvement
merged_all['Alcohol_and_Distraction'] = merged_all.apply(lambda x: 1 if x['A_POSBAC'] == 1 and x['A_DIST'] == 1 else 0, axis=1)

merged_all.head()

# Count the occurrences of each value in the 'Alcohol_and_Distraction' column
feature_counts = merged_all['Alcohol_and_Distraction'].value_counts()

# Create a bar chart to visualize the distribution
plt.figure(figsize=(8, 6))
feature_counts.plot(kind='bar', color='skyblue')
plt.title('Distribution of Alcohol and Distraction Involvement')
plt.xlabel('Alcohol and Distraction Involvement')
plt.ylabel('Count')
plt.xticks([0, 1], ['Not Involved', 'Involved'], rotation=0)
plt.show()

In [ ]:


#contributing factors
contributing_factors = ['A_POSBAC', 'A_DIST', 'A_RD', 'A_ROLL']  

# Labels for age groups in A_AGE3
age_group_3_labels = {
    1: "0-3", 2: "4-7", 3: "8-12", 4: "13-15", 5: "16-20", 6: "21-24", 7: "25-34", 
    8: "35-44", 9: "45-54", 10: "55-64", 11: "65-74", 12: "75+", 13: "Unknown"
}

# Aggregating total fatalities and contributing factors for each age groupe 
total_fatalities_age3 = merged_all.groupby('A_AGE3')['FATALS'].sum()
factors_age3 = merged_all.groupby('A_AGE3')[contributing_factors].sum()

# Adding labels to the data to make it more clear 
total_fatalities_age3.index = total_fatalities_age3.index.map(age_group_3_labels)
factors_age3.index = factors_age3.index.map(age_group_3_labels)

# Convert to DataFrame for plotting
total_fatalities_age3_df = total_fatalities_age3.reset_index()
total_fatalities_age3_df.columns = ['Age Group', 'Total Fatalities']
factors_age3_df = factors_age3.reset_index()

# Plotting total fatalities
plt.figure(figsize=(12, 6))
sns.barplot(x='Age Group', y='Total Fatalities', data=total_fatalities_age3_df, palette="viridis")
plt.title('Total Fatalities by Age Group (A_AGE3)')
plt.xlabel('Age Group')
plt.ylabel('Total Fatalities')
plt.xticks(rotation=45)
plt.show()


In [ ]:
merged_all.head()

## 5. Supervised Learning: Fatality Prediction Model

**Objective**: Predict the number of fatalities in traffic accidents using regional and temporal factors.

**Feature Selection Rationale**:
- **A_REGION**: Geographic regions show different traffic patterns, infrastructure quality, and emergency response capabilities
- **A_DOW**: Day of week captures different driving behaviors (commuter vs. recreational traffic)

**Model Choice**: Linear Regression
- Simple and interpretable model suitable for continuous target variable (fatality count)
- Baseline model to establish fundamental relationships before exploring complex algorithms

**Evaluation Method**: 10-fold Cross-Validation with Mean Squared Error
- Provides robust performance estimation across different data subsets
- MSE is appropriate for regression tasks with count data

In [ ]:
# Supervised Learning: Linear Regression for Fatality Prediction

# Select predictor variables based on domain knowledge
X = merged_all[['A_REGION', 'A_DOW']]

# Define target variable (number of fatalities)
y = merged_all['FATALS']

print(f"Features: {X.columns.tolist()}")
print(f"Target: FATALS (Number of fatalities)")
print(f"Dataset size: {X.shape[0]} samples, {X.shape[1]} features")

# One-hot encoding for categorical variables
column_transformer = ColumnTransformer([
    ("region_encoder", OneHotEncoder(), ["A_REGION"]),
    ("dow_encoder", OneHotEncoder(), ["A_DOW"])
], remainder='passthrough')

X_transformed = column_transformer.fit_transform(X)
print(f"Features after encoding: {X_transformed.shape[1]} dimensions")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Initialize and train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# 10-fold Cross-validation
cv_scores = cross_val_score(model, X_transformed, y, cv=10, scoring='neg_mean_squared_error')

# Convert to positive MSE scores
mse_scores = -cv_scores
cv_mean = mse_scores.mean()
cv_std = mse_scores.std()

print(f'\n10-Fold Cross-Validation Results:')
print(f'Mean MSE: {cv_mean:.4f}')
print(f'Standard Deviation: {cv_std:.4f}')
print(f'MSE Range: [{cv_mean - cv_std:.4f}, {cv_mean + cv_std:.4f}]')

# Visualize cross-validation results
plt.figure(figsize=(10, 6))
plt.errorbar(np.arange(1, 11), mse_scores, yerr=cv_std, fmt='o', capsize=5, capthick=2)
plt.title('10-Fold Cross-Validation MSE Scores for Linear Regression')
plt.xlabel('Fold Number')
plt.ylabel('Mean Squared Error')
plt.grid(True, alpha=0.3)
plt.show()

## 6. Unsupervised Learning: K-Means Clustering Analysis

**Objective**: Discover hidden patterns and group similar accidents to understand accident characteristics.

**Features Selected**: 
- **A_REGION**: Geographic clustering reveals regional accident patterns
- **FATALS**: Accident severity clustering identifies high-risk scenarios

**Algorithm Choice**: K-Means Clustering
- Well-suited for numerical data with clear geometric interpretation
- Computationally efficient for large datasets
- Provides interpretable centroids representing cluster characteristics

**Distance Metric**: Euclidean Distance
- Standard choice for K-means algorithm
- Appropriate for numerical features after standardization
- Captures both regional and severity similarities effectively

**Optimization Strategy**: Elbow Method
- Systematic approach to determine optimal number of clusters
- Balances model complexity with clustering quality

In [ ]:
# Unsupervised Learning: K-Means Clustering Analysis

# Select features for clustering analysis
cluster_data = merged_all[['A_REGION', 'FATALS']]
print(f"Clustering features: {cluster_data.columns.tolist()}")
print(f"Data shape: {cluster_data.shape}")

# Standardize features for clustering
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)
print("Features standardized for equal weighting in clustering")

# Elbow Method to determine optimal number of clusters
wcss = []  # Within-Cluster Sum of Squares
k_range = range(1, 11)

print("Computing WCSS for different cluster numbers...")
for i in k_range:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

# Plot Elbow Method results
plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, 'bo-', linewidth=2, markersize=8)
plt.title('Elbow Method for Optimal Number of Clusters', fontsize=14)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Within-Cluster Sum of Squares (WCSS)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(k_range)

# Highlight the elbow point (k=3 based on visual inspection)
plt.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Optimal k=3')
plt.legend()
plt.show()

# Apply K-Means with optimal number of clusters
optimal_clusters = 3
print(f"\nApplying K-Means with k={optimal_clusters} clusters")

kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(scaled_data)

# Add cluster labels to original dataset
merged_all['Cluster_Labels'] = cluster_labels

# Display cluster centroids (in original scale)
centroids_scaled = kmeans.cluster_centers_
centroids_original = scaler.inverse_transform(centroids_scaled)

print(f"\nCluster Centroids (Original Scale):")
print("Cluster | A_REGION | FATALS")
print("-" * 30)
for i, centroid in enumerate(centroids_original):
    print(f"   {i}    |   {centroid[0]:.2f}   |  {centroid[1]:.2f}")

# Cluster distribution
cluster_counts = merged_all['Cluster_Labels'].value_counts().sort_index()
print(f"\nCluster Distribution:")
for i in range(optimal_clusters):
    print(f"Cluster {i}: {cluster_counts[i]} accidents ({cluster_counts[i]/len(merged_all)*100:.1f}%)")

# Visualize clusters
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green', 'purple', 'orange']
for i in range(optimal_clusters):
    cluster_data_subset = merged_all[merged_all['Cluster_Labels'] == i]
    plt.scatter(cluster_data_subset['A_REGION'], cluster_data_subset['FATALS'], 
                c=colors[i], label=f'Cluster {i}', alpha=0.6)

# Plot centroids
for i, centroid in enumerate(centroids_original):
    plt.scatter(centroid[0], centroid[1], c='black', marker='x', s=200, linewidths=3)
    plt.annotate(f'C{i}', (centroid[0], centroid[1]), xytext=(5, 5), 
                textcoords='offset points', fontweight='bold')

plt.title('K-Means Clustering Results: Regional and Fatality Patterns')
plt.xlabel('A_REGION (Geographic Region)')
plt.ylabel('FATALS (Number of Fatalities)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Conclusions and Key Findings

### What I Learned from This Project

**Data Science Skills**:
- **Data Integration**: Successfully merged multiple large datasets with complex relationships
- **Feature Engineering**: Created meaningful variables from categorical accident data
- **Machine Learning**: Applied both supervised and unsupervised learning techniques effectively

**Domain Knowledge**:
- **Traffic Safety Patterns**: Discovered regional and temporal patterns in fatal accidents
- **Risk Factor Analysis**: Identified key contributing factors to accident severity
- **Policy Implications**: Generated insights relevant for traffic safety interventions

### Project Limitations

**Data Scope**:
- **Single Year Analysis**: Limited to 2021 data - temporal trends across years not captured
- **Fatal Accidents Only**: FARS data excludes non-fatal accidents, limiting generalizability
- **Feature Selection**: Limited feature set for modeling - additional variables could improve predictions

**Methodological Constraints**:
- **Simple Models**: Used baseline algorithms - more complex models might capture non-linear relationships
- **Regional Granularity**: State-level analysis may miss local traffic patterns and infrastructure differences

### Technical Challenges Faced

1. **Data Integration Complexity**: Merging three datasets with different granularities and relationships
2. **Missing Data Handling**: Systematic approach needed for columns with significant missing values
3. **Categorical Variable Encoding**: Proper transformation of FARS coded variables for machine learning
4. **Scale Differences**: Standardization required for clustering due to different feature ranges

### Algorithms and Model Selection

**Linear Regression**:
- **Why Chosen**: Interpretable baseline model for continuous target variable (fatality count)
- **Advantages**: Simple interpretation, fast computation, established statistical properties
- **Use Case**: Suitable for initial exploration of relationships between regional/temporal factors and fatalities

**K-Means Clustering**:
- **Why Chosen**: Efficient algorithm for identifying patterns in numerical data
- **Advantages**: Clear geometric interpretation, computationally scalable, provides interpretable centroids
- **Use Case**: Effective for discovering regional accident severity patterns

### Future Work Opportunities

**Enhanced Modeling**:
- **Advanced Algorithms**: Implement Random Forest, XGBoost, or neural networks for better prediction accuracy
- **Time Series Analysis**: Incorporate multi-year data to identify seasonal and yearly trends
- **Ensemble Methods**: Combine multiple models for improved robustness and accuracy

**Extended Feature Engineering**:
- **Weather Integration**: Include detailed weather data for environmental factor analysis
- **Road Infrastructure**: Add highway type, construction zones, and traffic signal data
- **Socioeconomic Factors**: Incorporate demographic and economic variables by region

**Real-World Applications**:
- **Predictive Dashboard**: Deploy model as real-time accident risk assessment tool
- **Policy Analysis**: Use insights for evidence-based traffic safety policy recommendations
- **Resource Allocation**: Guide emergency response resource placement based on predicted high-risk areas

## 8. References and Data Sources

### Primary Data Source
**National Highway Traffic Safety Administration (NHTSA)**. (2021). *Fatality Analysis Reporting System (FARS)*. 
- **Website**: https://www.nhtsa.gov/research-data/fatality-analysis-reporting-system-fars
- **Description**: Comprehensive nationwide census of fatal motor vehicle traffic crashes
- **Data Files Used**: ACC_AUX.CSV, VEH_AUX.CSV, PER_AUX.CSV (2021 dataset)

### Technical Documentation
**Python Libraries and Frameworks**:
- **Pandas**: McKinney, W. (2010). Data structures for statistical computing in Python. *Proceedings of the 9th Python in Science Conference*.
- **Scikit-learn**: Pedregosa, F., et al. (2011). Scikit-learn: Machine learning in Python. *Journal of Machine Learning Research*, 12, 2825-2830.
- **Matplotlib**: Hunter, J. D. (2007). Matplotlib: A 2D graphics environment. *Computing in Science & Engineering*, 9(3), 90-95.

### Methodological References
**Machine Learning Techniques**:
- **K-Means Clustering**: MacQueen, J. (1967). Some methods for classification and analysis of multivariate observations. *Proceedings of the Fifth Berkeley Symposium on Mathematical Statistics and Probability*.
- **Cross-Validation**: Stone, M. (1974). Cross-validatory choice and assessment of statistical predictions. *Journal of the Royal Statistical Society*, 36(2), 111-147.

### Traffic Safety Research Context
**FARS Data Applications**:
- **NHTSA Traffic Safety Facts**: Annual reports utilizing FARS data for policy development and safety analysis
- **Academic Research**: Numerous peer-reviewed studies using FARS data for transportation safety research

*This project demonstrates academic application of data science techniques to real-world transportation safety data for educational and portfolio purposes.*